## Extracción de datos y obtención de características

### Importación de bibliotecas

In [12]:
# Importación de bibliotecas
import cv2
import math
import time
import glob
import scipy.signal
import numpy as np
import pandas as pd
import mediapipe as mp
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

from matplotlib import pyplot as plt
%matplotlib inline

### Procesamiento de vídeo
Se realiza un procesado en los vídeos para extraer datos relevantes que sirvan para detectar el Parkinson.

#### Funciones para leer los vídeos y mostrar fotogramas

In [13]:
'''
Se obtienen los fotogramas de uno en uno del vídeo pasado por parámetro.

Parámetros:
 - vd: vídeo del que obtener los fotogramas.
 
Retorno:
 - frame: fotograma actual del vídeo.
'''
def frameVideo(vd):
    vc = cv2.VideoCapture(vd)

    if (vc.isOpened()==False):
        print("Error")
    else:
        while(vc.isOpened()):
            ret,frame = vc.read()
            if ret:
                yield frame
            else:
                vc.release()

In [14]:
'''
Se muestra el fotograma pasado por parámetro.

Parámetros:
 - frame: fotograma que se desea mostrar.
'''
def mostrarFrame(frame):
    plt.figure(figsize=(18, 16))
    frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    plt.imshow(frame)
    plt.title("Prueba")
    plt.show()

In [15]:
'''
Se obtiene el fotograma deseado del vídeo pasado por parámetro. 

Parámetros:
 - vid: vídeo del que obtener el fotograma.
 - nframe: la posición del fotograma que se desea obtener.
 
Retorno:
 - frame: fotograma deseado del vídeo.
'''
def obtenerFrame(vid,nframe):
    vc = cv2.VideoCapture(vid)

    if (vc.isOpened()==False):
        print("Error")
        return None
    else:
        for i in range(nframe):
            ret,frame = vc.read()
        
        return frame

In [16]:
'''
Se crea un objeto de vídeo para guardar los fotogramas pasados por parámetro.

Parámetros:
 - ruta: ruta donde se desea guardar el objeto de vídeo creado.
 - frames: fotogramas que se desean guardar.
 - encoder: codificador para guardar los fotogramas. Por defecto: cv2.VideoWriter_fourcc(*'DIVX').
 - fps: los fotogramas por segundo que tendrá el vídeo creado. Por defecto: 15. 
'''
def grabarFrames(ruta,frames,encoder=cv2.VideoWriter_fourcc(*'DIVX'),fps=15):
    for i in frames:
        if i is not None:
            h,z=i.shape[:2]
            break
    size=(z,h)
    out = cv2.VideoWriter(ruta,encoder,fps,size)
    
    for i in frames:
        out.write(i)
    
    out.release()

In [17]:
'''
Se redimensiona y se muestra, si se desea, el fotograma pasado por parámetro.

Parámetros:
 - frame: fotograma que se quiere redimensionar.
 - width: nueva anchura del fotograma. Por defecto: 480.
 - height: nueva altura del fotograma. Por defecto: 480.
 - show: valor booleano para indicar si se desea mostrar o no el fotograma redimensionado. Por defecto: False.
'''
def resize_and_show(frame,width=480,height=480,show=False):
    h, w = frame.shape[:2]
    if h < w:
        img = cv2.resize(frame, (width, math.floor(h/(w/width))))
    else:
        img = cv2.resize(frame, (math.floor(w/(h/height)), height))
    
    if show:
        mostrarFrame(frame)

#### Marcado de los puntos en la mano
Para cada fotograma del vídeo, se marcan los puntos de la mano. Esto servirá para conocer la posición de los dedos en cada insante del vídeo.

In [18]:
'''
Se marcan los puntos de la mano del fotograma pasado por parámetro.

Parámetros:
 - frame: fotograma con la mano en donde indicar los puntos.
 - static: modo de imagen estático. Por defecto: True.
 - max_num_hands: número máximo de manos. Por defecto: 1.
 - min_detection_confidence: mímina detección de confianza. Por defecto: 0.
 - show: valor booleano para indicar si se desea redimensionar y mostrar o no el fotograma. Por defecto: False.
 
Retorno:
 - annotated_frame: fotograma con la mano.
 - results.mulit_hand_landmarks: puntos de la mano.
'''
def frameMano(frame, static=True,max_num_hands=1,min_detection_confidence=0,show=False):
    with mp_hands.Hands(static_image_mode=static,max_num_hands=max_num_hands, min_detection_confidence=min_detection_confidence) as hands:
        results = hands.process(cv2.flip(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), 1))
        frame_hight, frame_width, _ = frame.shape
        annotated_frame = cv2.flip(frame.copy(), 1)
        
        if results.multi_hand_landmarks is None:
            return annotated_frame,None
        
        for hand_landmarks in results.multi_hand_landmarks:
            # Print index finger tip coordinates.
            print(
                f'Index finger tip coordinate: (',
                f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * frame_width}, '
                f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * frame_hight})'
            )
            mp_drawing.draw_landmarks(annotated_frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        if show:
            resize_and_show(cv2.flip(annotated_frame, 1))
        
        return annotated_frame,results.multi_hand_landmarks

#### Cálculo de las distancias
Se calculan las distancias para poder extraer las características. Sin embargo, estas distancias no son las definitivas.

In [19]:
'''
Se calcula la distancia entre el punto del dedo índice y el punto del dedo pulgar.

Parámetros:
 - mano: lista con los puntos de la mano.
 
Retorno:
 - np.linalg.norm(p4-p8): distancia entre los puntos.
'''
def calcularDistancia(mano):
    x4=mano[0].landmark[4].x
    y4=mano[0].landmark[4].y
    x8=mano[0].landmark[8].x
    y8=mano[0].landmark[8].y
    
    if x4<0 or y4<0 or x8<0 or y8<0:
        return None
    
    p4=np.array((x4,y4))
    p8=np.array((x8,y8))
    
    return np.linalg.norm(p4-p8)

In [20]:
'''
Se calcula la distancia de todos los puntos de los dedos índice y pulgar 
de la mano pasados por parámetro.

Parámetros:
 - manos: estructura con los puntos de la mano.
 
Retorno:
 - dist: lista de las distancias de los puntos de la mano.
'''
def calcularDistanciasVideo(manos):
    dist = []
    for i in manos:
        if i is not None:
            dist.append(calcularDistancia(i))
        
    return dist

### Limpieza de los datos
Normalmente, los datos extraídos no son perfectos, pues podrían contener ruido que los empeoren. En este caso, no hay excesivo ruido, pero es necesario aun así aplicar un filtrado.

#### Filtrado y representación de las distancias
Para cada vídeo, se realiza el cálculo de todas las distancias y se muestran en forma de gráfica.

Además, para suavizar un problema de la biblioteca en el que los puntos cambian sin haberse producido movimiento de la mano, se aplica el filtro de Savitzky–Golay. 

In [21]:
'''
Se obtienen la gráfica de las distancias obtenidas.

Parámetros:
 - ruta: lugar donde se encuentra el vídeo.
 - guardar: lugar donde guardar la gráfica. Por defecto: None.
 
Retorno:
 - dist: lista de las distancias del dedo índice al dedo pulgar. 
'''
def analizarVideo(ruta,guardar=None):
    manos=[]
    for i in frameVideo(ruta):
        frame,mano = frameMano(i)
        manos.append(mano)

    dist = calcularDistanciasVideo(manos)
    
    plt.style.use('seaborn-whitegrid')
    fig = plt.figure(figsize=(30, 20))
    ax = plt.axes()
    x = np.linspace(0,len(dist), len(dist))
    plt.plot(x, dist)
    plt.plot(x,scipy.signal.savgol_filter(dist, 3, 1))
    dist = scipy.signal.savgol_filter(dist, 3, 1)
    if guardar is not None:
        plt.savefig("../Outputs/"+guardar)
    return dist

### Obtención de datos
Se obtienen los datos de todos los vídeos para poder obtener características a partir de ellos.

In [1]:
inicio = time.time()
datos = []
for i in glob.glob("../TFG/Videos/*"):
    aux = i.split("/")[-1].split("\\")[-1]
    print("\n", aux, "\n")
    datos.append(analizarVideo(i,guardar="../TFG/Outputs/Graficas/"+aux+"v11.png"))
    
fin = time.time()
print("Tiempo total:", fin - inicio)

NameError: name 'time' is not defined

### Obtención de características
A partir de los datos obtenidos y de los proporcionados en el nombre de los vídeos, se extraen características que sirvan para poder indentificar el Parkinson.
Las características proporcionadas en el nombre de los vídeos son:
- Si la mano es la derecha o la izquierda.
- El sexo de la persona.
- La edad de la persona.

Las características calculadas son:
- Amplitud de la pinza.
- Tiempo que se tarda en abrir la pinza.
- Velocidad a la que se abre la pinza.

#### Obtención de las características proporcionadas
A partir del nombre de cada vídeo, se obtienen algunas de las características. Estas características están escritas como letras, pero se realiza una conversión a forma numérica, donde se obtiene:
- La mano de la persona:
    - Derecha (D) -> 0
    - Izquierda (I) -> 1
- Sexo de la persona:
    - Hombre (H) -> 0
    - Mujer (M) -> 1

La edad permanece con tipo numérico. Además, para saber si una persona tiene Parkinson o no, que también se recoge del nombre del vídeo, se añade una columna de forma numérica. Si la persona no tiene Parkinson, se simbolizará con el número 0, en caso opuesto, con el número 1. 

In [ ]:
manos_der_izq=[]
parkinson=[]
sexo=[]
edad=[]
for i in glob.glob("..\TFG\Videos\*"):
    aux = i.split("\\")[-1]
    if aux.split("_")[-1].split(" ")[0][0] == 'D':
        manos_der_izq.append(0)
    elif aux.split("_")[-1].split(" ")[0][0] == 'I':
        manos_der_izq.append(1)
    if aux.split(" ")[-1].split(".")[0].split("-")[0] == '(H':
        sexo.append(0)
    elif aux.split(" ")[-1].split(".")[0].split("-")[0] == '(M':
        sexo.append(1)
    edad.append(aux.split(" ")[-1].split(".")[0].split("-")[1].split(')')[0])
    if "CONTROL" in aux: 
        parkinson.append(0)
    else:
        parkinson.append(1)

#### Obtención de la amplitud
Se recogen los máximos y los mínimos de las 5 primeras aperturas y de las 5 últimas. A continuación, se calcula la diferencia que las separa. Esta diferencia será algo más real que la obtenida anteriormente, ya que se tiene en cuenta la distancia desde las yemas de los dedos en vez desde el centro de los dedos, donde está el punto de la biblioteca de Python. 

In [ ]:
diferencias=[]
maximos=[]
minimos=[]
j=-1
salir = False
for d in datos:
    i = 1
    minimo=[]
    maximo=[]
    diferencia=[]
    media = np.mean(d)
    j+=1
    for f in range(10):
        min_actual = float('inf')
        max_actual = 0
        if f == 5:
            i = -1
        while d[i] > media:
            if f == 5:
                i-=1
            else:
                i+=1
        while d[i] > media or not salir:
            if d[i] < media and d[i] < min_actual:
                min_actual = d[i] 
            if d[i] > media:
                salir = True
                if d[i] > max_actual:
                    max_actual = d[i]
            if f >= 5:
                i-=1
            else:
                i+=1
        salir = False
        
        minimo.append(min_actual)
        maximo.append(max_actual)
        diferencia.append(max_actual - min_actual)
        
    diferencias.append(diferencia)
    minimos.append(minimo)
    maximos.append(maximo)

#### Obtención del tiempo
Con los máximos y mínimos obtenidos en el paso anterior, se puede conocer cuántos fotogramas hay entre ambos, para conocer la velocidad. Para ello, se obtiene la posición que ocupan cada máximo y mínimo para poder restarlas y extraer el número de posiciones (i. e. fotogramas) que los separan.

In [ ]:
tiempos=[]
for m in range(len(minimos)):
    mini = []
    maxi = []
    for a in range(10):
        mini.append(list(datos[m]).index(minimos[m][a]))
        maxi.append(list(datos[m]).index(maximos[m][a]))

    resta = []
    for m1, m2 in zip(mini,maxi):
        resta.append(abs(m1-m2))
    tiempos.append(resta)

#### Obtención de la velocidad
Una vez obtenidas las características de amplitud y tiempo, se puede calcular la velocidad realizando la división de la amplitud realizada entre el tiempo que se ha tardado en realizarla.

Este cálculo se realiza directamente en la inserción de los datos en el dataframe, unas celdas más abajo. Esto es debido a que es importante que la velocidad tenga en cuenta la amplitud de la pinza normalizada.

#### Normalización de las amplitudes
Teniendo en cuenta que no todos los vídeos están grabados a la misma distancia de la cámara, las amplitudes deberán estar normalizadas para que todas estén al mismo nivel.

In [ ]:
'''
Se obtienen los datos de la lista normalizados con respecto al máximo y mínimo de cada vídeo.

Parámetros
 - lista: estructura con los datos que se van a normalizar.
 - maximo: punto máximo que se ha alcanzado.
 - minimo: punto mínimo que se ha alcanzado.
 
Retorno
 - norm: lista con los datos normalizados.
'''
def normalizacion(lista, maximo, minimo):
    norm = []
    p = 0
    for l in range(len(lista)):
        if l % 10 == 0 and l != 0:
            p+=1
        norm.append((lista[l] - minimo) / (maximo - minimo))
    return norm

#### Extracción de los datos
En este paso, las características se almacenarán en formato tabla utilizando dataframes, donde las filas serán las características de cada paciente y las columnas serán el nombre de cada característica. Las características son extraídas a dos archivos:
- CSV: que se utilizará para guardar los datos separados por comas para poder entrenar los modelos.
- XLSX: que se utilizará simplemente para poder visualizar los datos de una manera más clara.

Además, en la propia inserción de las características en el dataframe, se calculará la diferencia de las normalizaciones de máximos y mínimos y la velocidad de apertura en la pinza.

In [ ]:
# Se extraen las columnas deseadas de la lista con los datos.

'''
Parámetros:
 - lista: estructura con los datos que se desean extraer.
 - ruta: lugar donde se guardarán los ficheros con los datos extraídos.
 - columns: columnas que se desean extraer.
 - header: nombre de las columnas.
'''
def extraerDatos(df, ruta='../TFG/Outputs/Datos', columns = None):
    if columns == None:
        columns = ['Max1.', 'Max2.', 'Max3.', 'Max4.', 'Max5.', 'Max6.', 'Max7.', 'Max8.', 'Max9.', 'Max10.', 
         'Min1.', 'Min2.', 'Min3.', 'Min4.', 'Min5.', 'Min6.', 'Min7.', 'Min8.', 'Min9.', 'Min10.', 
         'Diff1.', 'Diff2.', 'Diff3.', 'Diff4.', 'Diff5.', 'Diff6.', 'Diff7.', 'Diff8.', 'Diff9.', 
         'Diff10.', 'Max1. norm.', 'Max2. norm.', 'Max3. norm.', 'Max4. norm.', 'Max5. norm.', 
         'Max6. norm.', 'Max7. norm.', 'Max8. norm.', 'Max9. norm.', 'Max10. norm.', 'Min1. norm.', 
         'Min2. norm.', 'Min3. norm.', 'Min4. norm.', 'Min5. norm.', 'Min6. norm.', 'Min7. norm.', 
         'Min8. norm.', 'Min9. norm.', 'Min10. norm.', 'Diff1. norm.', 'Diff2. norm.', 'Diff3. norm.', 
         'Diff4. norm.', 'Diff5. norm.', 'Diff6. norm.', 'Diff7. norm.', 'Diff8. norm.', 'Diff9. norm.', 
         'Diff10. norm.', 'Mean time', 'Mean speed', 'Hand R(0)/L(1)', 'Sex M(0)/W(1)', 'Age', 'Parkinson']
        
    df.to_csv(ruta + '/datos.csv', sep=';', columns = columns, index=False, encoding='utf-8')
    df.to_excel(ruta + '/datos.xlsx', columns = columns, index=False, encoding='utf-8')

In [ ]:
columnas = ['Max1.', 'Max2.', 'Max3.', 'Max4.', 'Max5.', 'Max6.', 'Max7.', 'Max8.', 'Max9.', 'Max10.', 
            'Min1.', 'Min2.', 'Min3.', 'Min4.', 'Min5.', 'Min6.', 'Min7.', 'Min8.', 'Min9.', 'Min10.', 
            'Diff1.', 'Diff2.', 'Diff3.', 'Diff4.', 'Diff5.', 'Diff6.', 'Diff7.', 'Diff8.', 'Diff9.', 
            'Diff10.', 'Max1. norm.', 'Max2. norm.', 'Max3. norm.', 'Max4. norm.', 'Max5. norm.', 
            'Max6. norm.', 'Max7. norm.', 'Max8. norm.', 'Max9. norm.', 'Max10. norm.', 'Min1. norm.', 
            'Min2. norm.', 'Min3. norm.', 'Min4. norm.', 'Min5. norm.', 'Min6. norm.', 'Min7. norm.', 
            'Min8. norm.', 'Min9. norm.', 'Min10. norm.', 'Diff1. norm.', 'Diff2. norm.', 'Diff3. norm.', 
            'Diff4. norm.', 'Diff5. norm.', 'Diff6. norm.', 'Diff7. norm.', 'Diff8. norm.', 'Diff9. norm.', 
            'Diff10. norm.', 'Mean time', 'Mean speed', 'Hand R(0)/L(1)', 'Sex M(0)/W(1)', 'Age', 'Parkinson']

df = pd.DataFrame(columns=columnas)
for i in range(32): 
    lista=[]
    norm=[]
    velocidades=[]
    lista.extend(maximos[i])
    lista.extend(minimos[i])
    lista.extend(diferencias[i])
    max_norm=normalizacion(maximos[i], sorted(maximos[i])[-1], sorted(minimos[i])[0])
    min_norm=normalizacion(minimos[i], sorted(maximos[i])[-1], sorted(minimos[i])[0])
    lista.extend(max_norm)
    lista.extend(min_norm)
    for n1,n2 in zip(max_norm, min_norm):
        norm.append(n1-n2)
    lista.extend(norm)
    lista.append(np.mean(tiempos[i]))
    for d,t in zip(norm, tiempos[i]):
        velocidades.append(d/t)
    lista.append(np.mean(velocidades))
    lista.append(manos_der_izq[i])
    lista.append(sexo[i])
    lista.append(edad[i])
    lista.append(parkinson[i])
    df2 = pd.DataFrame(data = [lista], columns=columnas)
    df = pd.concat([df,df2])

#extraerDatos(df, columns = ['Max1.', 'Min1.', 'Diff1.', 'Hand R(0)/L(1)'])
extraerDatos(df)
df